In [1]:
import argparse
import time
import tiktoken
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 1024,  # Context length
        "emb_dim": 768,          # Embedding dimension
        "n_heads": 12,           # Number of attention heads
        "n_layers": 12,          # Number of layers
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": False,        # Query-Key-Value bias
        "latent_dim" : 16
    }

In [24]:
class MultiHeadLatentAttention(nn.Module):
    def __init__(self, d_in, d_out, dropout, num_heads, qkv_bias=False, latent_dim=None):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim
        self.latent_dim = latent_dim if latent_dim is not None else max(16, d_out // 8)

        self.W_query =  nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_DKV = nn.Linear(d_in, self.latent_dim, bias=qkv_bias)
        self.W_UK = nn.Linear(self.latent_dim, self.d_out, bias=qkv_bias)
        self.W_UV = nn.Linear(self.latent_dim, self.d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)

        ####################################################
        # KV cache-related code
        self.register_buffer("cache_c_kv", None, persistent=False)
        self.ptr_current_pos = 0
        ####################################################

    def forward(self, x, use_cache=False):
        x_udv = None
        b, num_tokens, d_in = x.shape

        x_udv_new = self.W_DKV(x)

        ####################################################
        # KV cache-related
        if use_cache:
            if self.cache_c_kv is None:
                self.cache_c_kv = x_udv_new
            else:
                self.cache_c_kv = torch.cat([self.cache_c_kv, x_udv_new], dim=1)

            x_udv = self.cache_c_kv
        else:
            x_udv = x_udv_new
        ####################################################

        keys = self.W_UK(x_udv)  # Shape: (b, seq_length, d_out)
        values = self.W_UV(x_udv) # Shape: (b, seq_length, d_out)
        queries = self.W_query(x) # Shape: (b, num_tokens, d_out)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        _, seq_length, _ = keys.shape
        keys = keys.view(b, seq_length, self.num_heads, self.head_dim).transpose(1, 2).contiguous()
        values = values.view(b, seq_length, self.num_heads, self.head_dim).transpose(1, 2).contiguous()
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2).contiguous()

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        ########################################################################################
        #NEW

        Q_num_tokens = queries.shape[-2]
        K_num_tokens = keys.shape[-2]

        device = queries.device
        if use_cache:
            q_positions = torch.arange(
                self.ptr_current_pos,
                self.ptr_current_pos + Q_num_tokens,
                device=device,
                dtype=torch.long,
            )
            self.ptr_current_pos += Q_num_tokens
        else:
            q_positions = torch.arange(Q_num_tokens, device=device, dtype=torch.long)
            self.ptr_current_pos = 0
        k_positions = torch.arange(K_num_tokens, device=device, dtype=torch.long)
        mask_bool = q_positions.unsqueeze(-1) < k_positions.unsqueeze(0)
        ########################################################################################

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

    def reset_cache(self):
        self.cache_c_kv = None
        self.ptr_current_pos = 0

In [31]:
torch.manual_seed(123)
mhla = MultiHeadLatentAttention(768, 768, 0.1, 8, latent_dim = 16)
x = torch.randn((4, 16, 768))
mhla(x, use_cache=False)

tensor([[[-0.4875, -0.0411,  0.0817,  ...,  0.4159,  0.0751, -0.0362],
         [-0.3747,  0.0040, -0.1376,  ...,  0.1695,  0.0344,  0.1456],
         [-0.2558, -0.0190, -0.0046,  ..., -0.1732, -0.0314,  0.0327],
         ...,
         [-0.0434, -0.0430,  0.0536,  ..., -0.0190,  0.1086,  0.0185],
         [-0.0076, -0.0088,  0.0619,  ...,  0.0151,  0.0752,  0.0256],
         [-0.0299, -0.0438,  0.0467,  ..., -0.0277,  0.0474,  0.0263]],

        [[-0.1220,  0.0608,  0.0130,  ..., -0.1628,  0.1217,  0.1792],
         [-0.2831,  0.0726,  0.0782,  ...,  0.1233,  0.1931,  0.0833],
         [-0.1875,  0.0378,  0.0440,  ...,  0.1351,  0.1351,  0.0959],
         ...,
         [ 0.0017,  0.0045, -0.0118,  ...,  0.0411,  0.0340,  0.0081],
         [-0.0067,  0.0040,  0.0021,  ...,  0.0042,  0.0231, -0.0021],
         [ 0.0036,  0.0009, -0.0563,  ...,  0.0039,  0.0498, -0.0169]],

        [[ 0.0805,  0.0756, -0.2643,  ..., -0.1146,  0.0218,  0.1169],
         [ 0.0141,  0.1417, -0.2870,  ...,  0

In [32]:
torch.manual_seed(123)
mhla = MultiHeadLatentAttention(768, 768, 0.1, 8, latent_dim = 16)
x = torch.randn((4, 16, 768))
mhla(x, use_cache=True)


tensor([[[-0.4875, -0.0411,  0.0817,  ...,  0.4159,  0.0751, -0.0362],
         [-0.3747,  0.0040, -0.1376,  ...,  0.1695,  0.0344,  0.1456],
         [-0.2558, -0.0190, -0.0046,  ..., -0.1732, -0.0314,  0.0327],
         ...,
         [-0.0434, -0.0430,  0.0536,  ..., -0.0190,  0.1086,  0.0185],
         [-0.0076, -0.0088,  0.0619,  ...,  0.0151,  0.0752,  0.0256],
         [-0.0299, -0.0438,  0.0467,  ..., -0.0277,  0.0474,  0.0263]],

        [[-0.1220,  0.0608,  0.0130,  ..., -0.1628,  0.1217,  0.1792],
         [-0.2831,  0.0726,  0.0782,  ...,  0.1233,  0.1931,  0.0833],
         [-0.1875,  0.0378,  0.0440,  ...,  0.1351,  0.1351,  0.0959],
         ...,
         [ 0.0017,  0.0045, -0.0118,  ...,  0.0411,  0.0340,  0.0081],
         [-0.0067,  0.0040,  0.0021,  ...,  0.0042,  0.0231, -0.0021],
         [ 0.0036,  0.0009, -0.0563,  ...,  0.0039,  0.0498, -0.0169]],

        [[ 0.0805,  0.0756, -0.2643,  ..., -0.1146,  0.0218,  0.1169],
         [ 0.0141,  0.1417, -0.2870,  ...,  0

In [20]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [21]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [22]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [35]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadLatentAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            num_heads = cfg["n_heads"],
            dropout = cfg["drop_rate"],
            qkv_bias = cfg["qkv_bias"],
            latent_dim = cfg["latent_dim"],
            )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x, use_cache=False):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)

        # x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        ####################################################
        #  KV cache-related
        x = self.att(x, use_cache=use_cache)
        ####################################################

        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x


In [36]:
torch.manual_seed(123)
trb = TransformerBlock(cfg = GPT_CONFIG_124M)
x = torch.randn((4, 16, 768))
trb(x, use_cache=True)

tensor([[[ 0.4884, -0.5535,  0.8693,  ...,  1.7824,  1.0540, -2.6442],
         [ 0.5594, -1.2984,  2.3836,  ...,  1.4760,  1.0618, -0.2848],
         [ 0.2841, -0.9488,  0.4599,  ...,  1.9853, -0.2283,  0.4034],
         ...,
         [ 0.0541,  0.4883,  1.0370,  ...,  0.6804, -1.3578, -0.1327],
         [ 1.5739, -0.3282, -1.0375,  ...,  0.2705,  0.7439,  0.4705],
         [-1.1375,  1.6443, -0.6123,  ...,  1.2042, -0.1471,  0.4203]],

        [[ 0.9517, -0.1953,  1.6359,  ...,  0.5231, -0.4581,  1.1008],
         [-0.7337,  0.9436, -1.3198,  ...,  0.9200,  0.8286,  1.5573],
         [ 0.1246,  0.7043,  1.0642,  ..., -0.4129, -0.8991,  0.3662],
         ...,
         [ 1.6171,  0.7193, -1.8199,  ..., -1.2530, -1.2707, -0.8413],
         [-0.1360, -0.6636, -0.8448,  ..., -0.4091, -0.9891,  0.3846],
         [ 0.0416, -1.3906, -0.3378,  ...,  1.2605, -1.2222, -1.0058]],

        [[ 0.3212,  0.4953, -0.9053,  ...,  1.1529, -0.7419,  0.4639],
         [-1.0039,  0.2175, -0.5057,  ...,  1

In [37]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # self.trf_blocks = nn.Sequential(
        #    *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        ####################################################
        #  KV cache-related
        self.trf_blocks = nn.ModuleList(
            [TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.current_pos = 0
        ####################################################

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx, use_cache=False):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)

        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        ####################################################
        #  KV cache-related
        if use_cache:
            pos_ids = torch.arange(self.current_pos, self.current_pos + seq_len, device=in_idx.device, dtype=torch.long)
            self.current_pos += seq_len
        else:
            pos_ids = torch.arange(0, seq_len, device=in_idx.device, dtype=torch.long)
        pos_embeds = self.pos_emb(pos_ids).unsqueeze(0)
        ####################################################

        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)

        # x = self.trf_blocks(x)
        ####################################################
        # KV cache-related
        for blk in self.trf_blocks:
            x = blk(x, use_cache=use_cache)
        ####################################################

        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

    ####################################################
    # KV cache-related
    def reset_kv_cache(self):
        for blk in self.trf_blocks:
            blk.att.reset_cache()
        self.current_pos = 0
    ####################################################

In [38]:
start_context = "Hello, I am"
tokenizer = tiktoken.get_encoding("gpt2")
encoded = tokenizer.encode(start_context)

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device, dtype=torch.bfloat16)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (att): MultiHeadLatentAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_DKV): Linear(in_features=768, out_features=16, bias=False)
        (W_UK): Linear(in_features=16, out_features=768, bias=False)
        (W_UV): Linear(in_features=16, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
  )
  (final_norm)

In [39]:
encoded_tensor = torch.tensor(encoded, device=device).unsqueeze(0)
print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
print("\nInput text:", start_context)
print("Encoded input text:", encoded)
print("encoded_tensor.shape:", encoded_tensor.shape)

if torch.cuda.is_available():
    torch.cuda.synchronize()
start = time.time()



                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [40]:
out = model(encoded_tensor)
out

tensor([[[-0.0762, -0.2383,  0.4707,  ...,  0.1816,  0.2393, -0.1211],
         [ 0.1113,  0.5586, -0.6602,  ...,  0.6094, -0.0830, -0.1250],
         [-0.3965, -0.2305, -0.2715,  ...,  0.1826,  0.2715,  0.9023],
         [ 0.4707, -0.3457,  0.2637,  ...,  0.3477, -0.2832,  0.2246]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)